In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [2]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band2
  atmpro = mls
  band = 2
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [-2, -2, 0]
  ng_refs = [2, 3, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 540
  vmin = 340
  w_diffuse = [(1.66, 1.66), (1.8, 1.8, 1.8), (1.8, 1.66, 1.45, 1.45, 1.45, 1.45)]
  wgt = [(0.6, 0.6), (0.6, 0.6, 0.6), (0.6, 0.6, 0.6, 0.6, 0.6, 0.6)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.150506                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.077074
0.000750      2.0  0.150897                  2.0  0.078080
0.001052      3.0  0.154483                  3.0  0.081460
0.001476      4.0  0.157811                  4.0  0.084729
0.002070      5.0  0.160774                  5.0  0.087912
0.002904      6.0  0.163010                  6.0  0.090972
0.004074      7.0  0.164338                  7.0  0.093834
0.005714      8.0  0.164455                  8.0  0.096376
0.008015      9.0  0.163082                  9.0  0.098447
0.011243     10.0  0.159821                 10.0  0.099823
0.015771     11.0  0.154195                 11.0  0.100218
0.022122     12.0  0.145761                 12.0  0.099119
0.031031     13.0  0.133896                 13.0  0.095951
0.043528     14.0  0.118228                 14.0  0.090135
0.061057     15.0  0.107771                 15.0  0.088774
0.085645     16.0  0.118233                 16.0  0.105127
0.120136     17.0  0.142700                 17.0  0.133970
0.168516     18.0  0.163342                 18.0  0.158454
0.236378     19.0  0.179192                 19.0  0.174673
0.331549     20.0  0.192917                 20.0  0.184982
0.465100     21.0  0.209116                 21.0  0.199237
0.652400     22.0  0.235296                 22.0  0.232647
0.915100     23.0  0.259517                 23.0  0.269549
1.283650     24.0  0.231296                 24.0  0.248089
1.800600     25.0  0.173344                 25.0  0.186990
2.525700     26.0  0.134207                 26.0  0.139611
3.542800     27.0  0.110392                 27.0  0.111069
4.969550     28.0  0.094594                 28.0  0.093823
6.970850     29.0  0.083647                 29.0  0.082528
9.778100     30.0  0.075054                 30.0  0.071824
13.715850    31.0  0.066558                 31.0  0.062287
19.239350    32.0  0.057801                 32.0  0.054916
26.987250    33.0  0.051570                 33.0  0.050316
37.855300    34.0  0.048001                 34.0  0.046057
53.100050    35.0  0.043697                 35.0  0.040247
73.887500    36.0  0.038405                 36.0  0.035182
97.662500    37.0  0.035530                 37.0  0.033995
121.437500   38.0  0.034808                 38.0  0.034784
145.212500   39.0  0.032620                 39.0  0.033377
168.987500   40.0  0.032527                 40.0  0.033752
192.762500   41.0  0.057578                 41.0  0.059529
216.537500   42.0  0.167167                 42.0  0.170333
240.312500   43.0  0.326836                 43.0  0.315026
264.087500   44.0  0.500570                 44.0  0.466851
287.862500   45.0  0.661044                 45.0  0.626250
311.637500   46.0  0.695882                 46.0  0.638068
335.412500   47.0  0.710241                 47.0  0.637253
359.187500   48.0  0.739594                 48.0  0.687130
382.962500   49.0  0.778400                 49.0  0.744981
406.737500   50.0  0.822551                 50.0  0.788733
430.512500   51.0  0.866454                 51.0  0.830903
454.287500   52.0  0.888889                 52.0  0.866133
478.062500   53.0  0.901492                 53.0  0.901756
501.837500   54.0  0.910508                 54.0  0.928898
525.612500   55.0  0.914068                 55.0  0.940261
549.387500   56.0  0.911287                 56.0  0.934613
573.162500   57.0  0.902917                 57.0  0.922349
596.937500   58.0  0.923258                 58.0  0.947919
620.712500   59.0  0.920720                 59.0  0.962532
644.487500   60.0  0.890931                 60.0  0.956777
668.262500   61.0  0.840791                 61.0  0.922669
692.037500   62.0  0.776111                 62.0  0.853821
715.812500   63.0  0.703552                 63.0  0.757663
739.587500   64.0  0.628612                 64.0  0.650577
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -60.366689   0.000000 -60.366689        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -60.075015   0.000002   
6.244000e-04 2     -60.366695   0.000017 -60.366678 -60.075022   0.000015   
8.759000e-04 3     -60.366697   0.000024 -60.366673 -60.075024   0.000019   
1.228600e-03 4     -60.366701   0.000034 -60.366667 -60.075028   0.000026   
1.723400e-03 5     -60.366706   0.000048 -60.366657 -60.075033   0.000036   
2.417400e-03 6     -60.366712   0.000068 -60.366644 -60.075040   0.000051   
3.390900e-03 7     -60.366722   0.000096 -60.366625 -60.075050   0.000071   
4.756500e-03 8     -60.366735   0.000136 -60.366599 -60.075063   0.000100   
6.672000e-03 9     -60.366753   0.000191 -60.366561 -60.075082   0.000141   
9.358900e-03 10    -60.366778   0.000269 -60.366510 -60.075109   0.000199   
1.312780e-02 11    -60.366814   0.000375 -60.366438 -60.075146   0.000281   
1.841450e-02 12    -60.366863   0.000522 -60.366342 -60.075199   0.000396   
2.583020e-02 13    -60.366934   0.000721 -60.366214 -60.075273   0.000558   
3.623230e-02 14    -60.367036   0.000987 -60.366049 -60.075379   0.000782   
5.082340e-02 15    -60.367183   0.001339 -60.365844 -60.075532   0.001090   
7.129060e-02 16    -60.367399   0.001816 -60.365583 -60.075751   0.001524   
1.000000e-01 17    -60.367699   0.002518 -60.365181 -60.076054   0.002185   
1.402710e-01 18    -60.368094   0.003594 -60.364500 -60.076452   0.003222   
1.967600e-01 19    -60.368597   0.005190 -60.363407 -60.076957   0.004788   
2.759970e-01 20    -60.369210   0.007485 -60.361725 -60.077567   0.007037   
3.871000e-01 21    -60.369898   0.010712 -60.359186 -60.078242   0.010147   
5.431000e-01 22    -60.370540   0.015218 -60.355322 -60.078853   0.014440   
7.617000e-01 23    -60.370723   0.021494 -60.349229 -60.078969   0.020581   
1.068500e+00 24    -60.369588   0.029790 -60.339797 -60.077703   0.029112   
1.498800e+00 25    -60.366953   0.038946 -60.328008 -60.074891   0.038947   
2.102400e+00 26    -60.363544   0.047930 -60.315614 -60.071261   0.048688   
2.949000e+00 27    -60.359587   0.057432 -60.302154 -60.067044   0.058474   
4.136600e+00 28    -60.355091   0.068467 -60.286625 -60.062231   0.069288   
5.802500e+00 29    -60.350053   0.082095 -60.267958 -60.056820   0.082394   
8.139200e+00 30    -60.344352   0.099548 -60.244804 -60.050659   0.099079   
1.141700e+01 31    -60.337827   0.122164 -60.215662 -60.043586   0.119896   
1.601470e+01 32    -60.330702   0.151289 -60.179413 -60.035829   0.146067   
2.246400e+01 33    -60.323986   0.188730 -60.135255 -60.028513   0.180709   
3.151050e+01 34    -60.318431   0.238439 -60.079992 -60.022492   0.228614   
4.420010e+01 35    -60.314177   0.306339 -60.007838 -60.017999   0.293360   
6.200000e+01 36    -60.313682   0.397979 -59.915703 -60.017758   0.377989   
8.577500e+01 37    -60.322900   0.515358 -59.807543 -60.028123   0.487447   
1.095500e+02 38    -60.340077   0.632599 -59.707478 -60.047110   0.602184   
1.333250e+02 39    -60.362521   0.753074 -59.609447 -60.071966   0.725014   
1.571000e+02 40    -60.392345   0.874765 -59.517579 -60.104600   0.851658   
1.808750e+02 41    -60.432254   1.006281 -59.425973 -60.147176   0.989300   
2.046500e+02 42    -60.492927   1.229112 -59.263815 -60.210954   1.220747   
2.284250e+02 43    -60.610825   1.817804 -58.793020 -60.336208   1.825762   
2.522000e+02 44    -60.802606   2.930060 -57.872547 -60.539748   2.916608   
2.759750e+02 45    -61.086455   4.623673 -56.462782 -60.836949   4.528746   
2.997500e+02 46    -61.482982   6.881911 -54.601071 -61.246874   6.702574   
3.235250e+02 47    -61.951168   9.309923 -52.641245 -61.727136   8.980025   
3.473000e+02 48    -62.471948  11.830968 -50.640980 -62.259770  11.307551   
3.710750e+02 49    -63

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')